##Requirements
Before running this notebook make sure you have the following:
- An Azure ML workspace
- A compute instance

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
from azureml.data.dataset_type_definitions import PromoteHeadersBehavior
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model


import requests
import json
import pandas as pd
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FLMPEZW5L to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-162632
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-162632


## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [3]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset
A Dataset is created from the csv file in the folder "data".

In [4]:
# Create a dataset from delimited files with header

#Retrieves default Datastore and display info
datastore = ws.get_default_datastore()
print("Default Datastore Info:\n") 
print(datastore)

#Google Drive path to dl csv
url_data = "https://docs.google.com/spreadsheets/d/1X9M3eNuBDv0ZKsOdidkdaBx9W1NubNDz3kxXuULsXmo/export?format=csv"

#Loads into Dataframe
df = pd.read_csv(url_data)

#Cleaning / Transform
df.drop("vintage",axis=1) # nor relevance for prediction was just needed to match weather with history

#Dictionary to rearrange rankings
#points

#Save into Datastore AND Register as dataset
dataset = Dataset.Tabular.register_pandas_dataframe(df,datastore,'Dataset_Wine',description="A Dataset composed of Tuscan wines with ratings, vintage, grapes and corresponding season avg weather")

#dataset = Dataset.Tabular.from_delimited_files(path=url_data,validate=True, include_path=False,header=PromoteHeadersBehavior.ONLY_FIRST_FILE_HAS_HEADERS, support_multi_line=False)

datasetWineTuscan = dataset.register(workspace=ws, name='Dataset_Wine', description="A Dataset composed of Tuscan wines with ratings, vintage, grapes and corresponding season avg weather")




#----Previous exploration to upload file from within Azure

directory = os.getcwd()
print("\nCurrent Working Directory:") 
print(directory)
#datastore.upload(src_dir='./data',
#                  target_path='datasets/',
#                  overwrite=True)

Default Datastore Info:

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-7076e1a5-ed45-4f08-ab0f-3509ecefc07e",
  "account_name": "mlstrg162632",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/fe14d4fd-0456-4a10-94fe-80f547dc32c6/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.

Current Working Directory:
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162632/code


In [5]:
#Display top 10 lines
dataset.take(10).to_pandas_dataframe()



,vintage,points,variety,winery,avg winter temp,avg spring temp,avg summer temp,avg fall temp,avg winter sun hour,avg spring sun hour,avg summer sun hour,avg fall sun hour,avg daily precip winter,avg daily precip spring,avg daily precip summer,avg daily precip fall
0,2010,85,Sangiovese,Casa Raia,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
1,2010,89,Sangiovese,Molinari Carlo,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
2,2010,92,Sangiovese,Castelli del Grevepesa,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
3,2010,93,Sangiovese,Le Gode,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
4,2010,97,Cabernet Franc,Le Macchiole,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
5,2010,90,Red Blend,Toscolo,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
6,2010,88,Sangiovese Grosso,Tenute Silvio Nardi,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
7,2010,88,Sangiovese,Mocali,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
8,2010,88,Chardonnay,Vigliano,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
9,2010,93,Red Blend,Boscarelli,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538


In [6]:

#Display Basic Stats
df.describe()

,vintage,points,avg winter temp,avg spring temp,avg summer temp,avg fall temp,avg winter sun hour,avg spring sun hour,avg summer sun hour,avg fall sun hour,avg daily precip winter,avg daily precip spring,avg daily precip summer,avg daily precip fall
count,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000
mean,2011.801858,88.643963,5.348069,14.564612,25.121155,16.391964,7.019741,12.121393,14.146674,9.088002,1.337985,1.339014,1.205760,1.202004
std,1.464112,2.355963,0.944459,0.863187,0.751995,0.749759,0.351382,0.302664,0.050172,0.341202,0.529709,0.319310,0.406899,0.362218
min,2010.000000,83.000000,4.661111,13.250000,24.184783,15.082418,6.451111,11.748913,14.097826,8.789011,0.557143,0.980435,0.605435,0.586813
25%,2011.000000,87.000000,4.879121,13.913043,24.331522,16.241758,6.927778,11.759783,14.097826,8.813187,1.248889,1.102174,1.076087,1.263736
50%,2012.000000,88.000000,5.188889,14.951087,25.271739,16.631868,7.048889,12.326087,14.118478,8.967033,1.307778,1.282609,1.247826,1.289011
75%,2013.000000,90.000000,5.250000,15.304348,25.918478,16.736264,7.261111,12.343478,14.165217,9.002198,1.730000,1.720652,1.446739,1.461538
max,2015.000000,97.000000,8.005556,15.494565,25.956522,17.467033,7.456044,12.436957,14.240217,9.669231,2.060000,1.739130,1.726087,1.497802


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":3,
    "validation_size":0.20,
    "label_column_name":'points',
    "primary_metric":'accuracy',
    "experiment_timeout_minutes":45  
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    training_data= dataset,
    compute_target = cluster,
    **automl_settings
    )


#metric = normalized_root_mean_squared_error => recommended for review score prediction // for regression scenario

#AUC_weighted for classification

In [8]:
# TODO: Submit your experiment
runAutoML = experiment.submit(automl_config, show_output=False)


Submitting remote run.
No run_configuration provided, running on myCluster with default configuration
Running on remote compute: myCluster


Experiment,Id,Type,Status,Details Page,Docs Page
Tuscan_Wine_Forecast,AutoML_8f8b3cc3-ac6f-408c-b718-00c542a238d7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+==========================

{'runId': 'AutoML_8f8b3cc3-ac6f-408c-b718-00c542a238d7',
 'target': 'myCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-11-02T19:47:43.345221Z',
 'endTimeUtc': '2021-11-02T20:06:35.390037Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'myCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Tuscan_Wine_Forecast","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-162632","workspace_name":"quick-starts-ws-162632","region":"southcentralus","compute_target":"myCluster",

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
RunDetails(runAutoML).show()
runAutoML.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_8f8b3cc3-ac6f-408c-b718-00c542a238d7',
 'target': 'myCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-11-02T19:47:43.345221Z',
 'endTimeUtc': '2021-11-02T20:06:35.390037Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'myCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Tuscan_Wine_Forecast","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-162632","workspace_name":"quick-starts-ws-162632","region":"southcentralus","compute_target":"myCluster",

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
#WARNING:root:The model you attempted to retrieve requires 'azureml-train-automl-runtime' to be installed at '==1.34.1'. 
#Please install 'azureml-train-automl-runtime==1.34.1' (e.g. `pip install azureml-train-automl-runtime==1.34.1`) and then rerun the previous command.

import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'azureml-train-automl-runtime==1.34.1'])


In [29]:
best_run, fitted_model = runAutoML.get_output()

runAutoML.wait_for_completion()
print(best_run)

RunDetails(best_run).show()

Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-core, training version:1.35.0, current version:1.34.0
Package:azureml-train-restclients-hyperdrive, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0
Pack

Run(Experiment: Tuscan_Wine_Forecast,
Id: AutoML_8f8b3cc3-ac6f-408c-b718-00c542a238d7_30,
Type: azureml.scriptrun,
Status: Completed)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [30]:
#TODO: Save the best model

# Register run as Model

model_name = "AleaumeModelAutoML"
description = "Best AutoML Model"
model_path ="outputs/modelAutoML.pkl"

#model = best_run.register_model(model_name = model_name, description = description, model_path = model_path )

model = runAutoML.register_model(model_name = model_name, description = description)

#SOURCE / HELP: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

#Unfortunately so far unsuccessful to register the model via SDK and get a default generated swagger.json uri. Also,  MSFT support recommends to do so via AML studio...

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [ ]:
#Register Model as ONNX model

#modelOnnx = model.register(workspace=ws,
                       #model_name='AleaumeModelAutoMLOnnx',                  # Name of the registered model in your workspace.
                       #model_path='outputs/modelAutoML.onnx',              # Local ONNX model to upload and register as a model.
                       #model_framework=Model.Framework.ONNX ,      # Framework used to create the model.
                       #model_framework_version='1.3',              # Version of ONNX used to create the model.
                       #description='Onnx model Best AutomL')

#print('Name:', modelOnnx.name)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [38]:
#To create a local deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=9064)

#Create Environment

env = Environment(name="AzureML-AutoML")
#myenv=env.clone("myenv")
#myenv.python.conda_dependencies.add_pip_package("joblib==1.1.0")

#Uploaded manually entry scripts in the below defined "entry_script" path.

my_inference_config = InferenceConfig(
    environment=env,
    source_directory= './',
    entry_script="./Users/odl_user_162632/score.py"
    #entry_script="./score.py"
)



# Deploy the service locally

service = model.deploy(ws, "local-service", [model], my_inference_config, deployment_config)
service.reload()
print(service.get_logs())

print(service.scoring_uri)

service.wait_for_deployment(show_output=True)


Generating Docker build context.


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_744753b4f832c74a6086091ff294e4a6
 ---> 7101fb6a62f0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> bdf331b377a5
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijk3NmVlMTc0LTM4ODItNDcyMS1iOTBhLWI1ZmVmNmI3MmYyNCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MjYzMiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MjYzMiIsIndvcmtzcGFjZUlkIjoiNzA3NmUxYTUtZWQ0NS00ZjA4LWFiMGYtMzUwOWVjZWZjMDdlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 15018e9e7ea4


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


 ---> ccc10bf9ac62
Step 4/5 : RUN mv '/var/azureml-app/tmp9n_eu6__.py' /var/azureml-app/main.py
 ---> Running in 6ed0f98dcbc3
 ---> dc4f08a643e9
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in d591d35b780e
 ---> 27557a1dac45
Successfully built 27557a1dac45
Successfully tagged local-service:latest
Container (name:musing_hoover, id:70175361fbea127445074488449db29f6d230b280e001c597a9aab9f5afd42bc) cannot be killed.
Container has been successfully cleaned up.
Image sha256:56fe3f74e6483c5782a7d39254f7113446bc7d0906f414f91b5161c5f519d5f3 successfully removed.
Starting Docker container...
Docker container running.
Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
2021-11-02T22:15:48,056700185+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-11-02T22:15:48,057492651+00:00 - rsyslog/run 
2021-11-02T22:15:48,060542807+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_705720c76ff57b57c77d57

In [18]:
#Call model to test 

#service.update(enable_app_insights=True)


uri = service.scoring_uri
requests.get("http://localhost:9064")
headers = {"Content-Type": "application/json"}
data = {
            "vintage":2013,
			"variety":"Sangiovese",
            "winery":"Casa Raia",
            "avg winter temp":4.2,
            "avg spring temp":15,
            "avg summer temp":27,
			"avg fall temp":16,
			"avg winter sun hour":7,
			"avg spring sun hour":12,
			"avg summer sun hour":16,
			"avg fall sun hour":9,
			"avg daily precip winter":3,
			"avg daily precip spring":3,
			"avg daily precip summer":1,
			"avg daily precip fall":2
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{'data': "'data'", 'message': 'Failed to predict '}


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [19]:
#Deploy to ACI

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, auth_enabled=True
)

service = model.deploy(
    ws,
    "mywebservice",
    [model],
    my_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads



2021-11-01 17:09:22+00:00 Creating Container Registry if not exists.
2021-11-01 17:09:23+00:00 Registering the environment.
2021-11-01 17:09:23+00:00 Use the existing image.
2021-11-01 17:09:24+00:00 Submitting deployment to compute.
2021-11-01 17:09:28+00:00 Checking the status of deployment mywebservice.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


.
2021-11-01 17:12:07+00:00 Checking the status of inference endpoint mywebservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-11-01T17:11:48,692384900+00:00 - iot-server/run 
2021-11-01T17:11:48,703359400+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-11-01T17:11:48,711208100+00:00 - rsyslog/run 
2021-11-01T17:11:48,771405000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_705720c76ff57b57c77d577152dabb18/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-11-01T17:11:49,105333400+00:00 - iot-server/finish 1 0
2021-11-01T17:11:49,109753400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (69)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 99
SPARK_HOME not set. Skipping PySpark Initialization.
Initializ

TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import requests
import json

primary, secondary = service.get_keys()

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {
			"vintage":2013,
			"variety": "Sangiovese",
			"winery": "Casa Raia",
			"avg winter temp": 4.2,
			"avg spring temp": 15,
			"avg summer temp": 27,
			"avg fall temp":16,
			"avg winter sun hour":7,
			"avg spring hour":12,
			"avg summer sun hour":16,
			"avg fall sun hour":9,
			"avg daily precip winter":3,
			"avg daily precip spring":3,
			"avg daily precip summer":1,
			"avg daily precip fall":2
	
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

{"data": "'data'", "message": "Failed to predict "}


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Deleting the webservice

service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
